In [ ]:
import tqdm
import numpy as np
import scipy.stats
import sklearn
import sklearn.cluster
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set_style('ticks')
palette = sns.color_palette("RdBu_r", n_colors=25)

In [ ]:
b = 100
n = 3000
cov = np.array([[1, 0.5], [0.5, 1]])

XY = scipy.stats.multivariate_normal(mean=[0, 0], cov=cov).rvs(n)
X, Y = np.split(XY, [1], axis=1)
XY_ = np.array([[x, y] for x in X for y in Y]).reshape(n**2, 2)
UV = sklearn.cluster.KMeans(b).fit(XY).cluster_centers_
U, V = np.split(UV, [1], axis=1)

print(XY.shape)
print(XY_.shape)
print(UV.shape)

In [ ]:
plt.scatter(X, Y)
plt.scatter(U, V, c='r', marker='x')

In [ ]:
def norm(mean, sigma):
    cov = np.eye(len(mean)) * sigma
    return lambda x: scipy.stats.multivariate_normal.pdf(x, mean=mean, cov=cov)

sigma = 1
bases = [norm(uv, sigma) for uv in UV]

In [ ]:
bases[0](XY_)

In [ ]:
phi = np.array([b(XY) for b in bases])
H = np.einsum('in,jn->ij', phi, phi) / n
h = np.mean([b(XY_) for b in bases], axis=1)

In [ ]:
lamb = 0.01
alpha = np.linalg.solve(H + lamb * np.eye(b), h)
alpha[alpha < 0] = 0

In [ ]:
np.mean(np.log(np.sum(alpha.reshape(-1, 1) * phi, axis=0)))

In [ ]:
-0.5 * np.log(1 - cov[0, 1]**2 / cov[0, 0] / cov[1, 1])